In [1]:
import streamlit as st
import pandas as pd
import numpy as np
import pyvista as pv
import os
import utils
from stpyvista import stpyvista
from datetime import datetime
import importlib

c:\Users\t-ehm\AppData\Local\Programs\Python\Python311\Lib\site-packages\stpyvista\__init__.py:18: UserWarning: Using Panel interactively in VSCode notebooks requires the jupyter_bokeh package to be installed. You can install it with:

   pip install jupyter_bokeh

or:
    conda install jupyter_bokeh

and try again.
  pn.extension("vtk", sizing_mode="stretch_both")


In [2]:
df = pd.concat([pd.read_csv('hka-aqm-am/' + f.removeprefix('._'), skiprows=1, sep=';', engine='python') for f in os.listdir('hka-aqm-am/')])
df.shape


(608036, 18)

In [16]:
importlib.reload(utils)
for y_feature in ['CO2', 'hum', 'tmp', 'vis', 'VOC']:
    aggregation_level = 'quarter_hour'
    model, scaler, rmse, mae, mape = utils.create_multivariate_transformer_model_for_feature(df, d_model=128, nhead=8, num_layers=4, dropout=0.1, batch_size=512, learning_rate=0.00031, epochs=20, y_feature=y_feature, aggregation_level=aggregation_level)

training data cutoff:  2023-07-15 02:45:00


c:\Studium\Semester_6\IoT_Project\utils.py:911: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train[columns_to_scale] = scaler.fit_transform(df_train[columns_to_scale])
c:\Studium\Semester_6\IoT_Project\utils.py:912: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test[columns_to_scale] = scaler.transform(df_test[columns_to_scale])
c:\Studium\Semester_6\IoT_Project\utils.py:916: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentati

tmp                         float64
hum                         float64
CO2                         float64
VOC                         float64
vis                         float64
IR                          float64
WIFI                        float64
BLE                         float64
rssi                        float64
channel_rssi                float64
channel_index               float64
spreading_factor            float64
bandwidth                   float64
f_cnt                       float64
isHoliday                   float64
isExamTime                  float64
weekday                     float64
month                       float64
hour_sin                    float64
semester_SS23               float64
semester_WS22/23            float64
semester_WS23/24            float64
group                       float64
device_id_hka-aqm-am001     float64
device_id_hka-aqm-am002     float64
device_id_hka-aqm-am003a    float64
device_id_hka-aqm-am003b    float64
device_id_hka-aqm-am004     

c:\Users\t-ehm\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
c:\Users\t-ehm\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


         actual   predicted
0    466.999999  507.917490
1    455.999999  511.247045
2    455.999999  497.097126
3    437.999999  491.931977
4    435.000001  467.129341
..          ...         ...
507  449.000001  452.418577
508  445.000002  460.463943
509  457.999999  459.616637
510  441.999999  473.774526
511  444.000001  461.024506

[512 rows x 2 columns]
Epoch 1/20, Training Loss: 0.3396
Epoch 1/20, Validation Loss: 0.0724
         actual   predicted
0    466.999999  514.766013
1    455.999999  511.370191
2    455.999999  495.964425
3    437.999999  487.313577
4    435.000001  468.542723
..          ...         ...
507  449.000001  474.936236
508  445.000002  477.902228
509  457.999999  475.026242
510  441.999999  486.233993
511  444.000001  474.150770

[512 rows x 2 columns]
Epoch 2/20, Training Loss: 0.2429
Epoch 2/20, Validation Loss: 0.1063
         actual   predicted
0    466.999999  475.073285
1    455.999999  474.116854
2    455.999999  462.390160
3    437.999999  455.989527


...

        actual  predicted
0    18.000003  62.833288
1    19.999996  62.987896
2    23.000004  92.505527
3    31.000000  58.724376
4    32.000002  98.735047
..         ...        ...
507  18.000003  27.633315
508  19.000006  25.282905
509  22.000001  21.659027
510  23.000004  28.350865
511  27.000002  55.471840

[512 rows x 2 columns]
Epoch 1/20, Training Loss: 0.6278
Epoch 1/20, Validation Loss: 0.3158
        actual   predicted
0    18.000003   30.375250
1    19.999996   29.669560
2    23.000004   55.853946
3    31.000000   15.319418
4    32.000002   43.382568
..         ...         ...
507  18.000003   48.934176
508  19.000006   52.556686
509  22.000001   57.522544
510  23.000004   81.916407
511  27.000002  137.627730

[512 rows x 2 columns]
Epoch 2/20, Training Loss: 0.5262
Epoch 2/20, Validation Loss: 0.2274
        actual   predicted
0    18.000003    1.959164
1    19.999996    6.030810
2    23.000004    9.080251
3    31.000000    3.357979
4    32.000002   13.441727
..         ...

c:\Studium\Semester_6\IoT_Project\utils.py:911: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train[columns_to_scale] = scaler.fit_transform(df_train[columns_to_scale])
c:\Studium\Semester_6\IoT_Project\utils.py:912: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test[columns_to_scale] = scaler.transform(df_test[columns_to_scale])
c:\Studium\Semester_6\IoT_Project\utils.py:916: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentati

tmp                         float64
hum                         float64
CO2                         float64
VOC                         float64
vis                         float64
IR                          float64
WIFI                        float64
BLE                         float64
rssi                        float64
channel_rssi                float64
channel_index               float64
spreading_factor            float64
bandwidth                   float64
f_cnt                       float64
isHoliday                   float64
isExamTime                  float64
weekday                     float64
month                       float64
hour_sin                    float64
semester_SS23               float64
semester_WS22/23            float64
semester_WS23/24            float64
group                       float64
device_id_hka-aqm-am001     float64
device_id_hka-aqm-am002     float64
device_id_hka-aqm-am003a    float64
device_id_hka-aqm-am003b    float64
device_id_hka-aqm-am004     

c:\Users\t-ehm\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
c:\Users\t-ehm\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


          actual   predicted
0    1132.999985  877.954182
1     954.999998  960.325586
2     990.000007  746.034679
3     811.000001  771.207534
4     739.000000  588.825468
..           ...         ...
507  1017.000004  834.534656
508  1033.999992  767.245761
509  1121.999998  775.217987
510  1019.999995  844.230736
511  1023.000005  732.112472

[512 rows x 2 columns]
Epoch 1/20, Training Loss: 0.1510
Epoch 1/20, Validation Loss: 0.1695
          actual   predicted
0    1132.999985  912.938098
1     954.999998  974.630389
2     990.000007  794.167024
3     811.000001  826.347546
4     739.000000  649.138048
..           ...         ...
507  1017.000004  878.117356
508  1033.999992  818.650986
509  1121.999998  835.079911
510  1019.999995  908.594551
511  1023.000005  801.694165

[512 rows x 2 columns]
Epoch 2/20, Training Loss: 0.0788
Epoch 2/20, Validation Loss: 0.1307
          actual    predicted
0    1132.999985   950.759436
1     954.999998  1014.300542
2     990.000007   817.679

In [3]:
importlib.reload(utils)
performance_df = pd.read_csv('model_performances.csv')
d_model=256
nhead=8
num_layers=4
dropout_pe=0.2
dropout_encoder=0.2
batch_size=1024
learning_rate=0.00031
epochs=25
for aggregation_level in ['quarter_hour', 'hour', 'half_hour']:
    for y_feature in ['CO2', 'hum', 'tmp', 'vis', 'VOC']:
        if aggregation_level == 'quarter_hour' and y_feature == 'CO2':
            # have to restart and dont want to train this model again
            continue
        model, scaler, rmse, mae, mape, train_loss, val_loss = utils.create_multivariate_transformer_model_for_feature(df, d_model=d_model, nhead=nhead, num_layers=num_layers, dropout_pe=dropout_pe, dropout_encoder=dropout_encoder, batch_size=batch_size, learning_rate=learning_rate, epochs=epochs, y_feature=y_feature, aggregation_level=aggregation_level)
        performance_df = performance_df.append({'model_name': 'multivariate_transformer','aggregation_level': aggregation_level, 'y_feature': y_feature, 'rmse': rmse, 'mae': mae, 'mape': mape, 'd_model': d_model, 'nhead': nhead, 'num_layers': num_layers, 'dropout_pe': dropout_pe,'dropout_encoder': dropout_encoder, 'batch_size': batch_size, 'learning_rate': learning_rate, 'epochs': epochs, 'train_loss': train_loss, 'val_loss': val_loss, 'note': '2 decoder Linear layers with LeakyReLu'}, ignore_index=True)
        performance_df.to_csv('model_performances.csv', index=False)


training data cutoff:  2023-07-15 02:45:00


c:\Studium\Semester_6\IoT_Project\utils.py:924: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train[columns_to_scale] = scaler.fit_transform(df_train[columns_to_scale])
c:\Studium\Semester_6\IoT_Project\utils.py:925: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test[columns_to_scale] = scaler.transform(df_test[columns_to_scale])
c:\Studium\Semester_6\IoT_Project\utils.py:929: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentati

tmp                         float64
hum                         float64
CO2                         float64
VOC                         float64
vis                         float64
IR                          float64
WIFI                        float64
BLE                         float64
rssi                        float64
channel_rssi                float64
channel_index               float64
spreading_factor            float64
bandwidth                   float64
f_cnt                       float64
isHoliday                   float64
isExamTime                  float64
weekday                     float64
month                       float64
hour_sin                    float64
semester_SS23               float64
semester_WS22/23            float64
semester_WS23/24            float64
group                       float64
device_id_hka-aqm-am001     float64
device_id_hka-aqm-am002     float64
device_id_hka-aqm-am003a    float64
device_id_hka-aqm-am003b    float64
device_id_hka-aqm-am004     

c:\Users\t-ehm\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
c:\Users\t-ehm\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\t-ehm\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "
c:\Users\t-eh

KeyboardInterrupt: 

In [26]:
all_model_files = [f for f in os.listdir('models/')]
all_model_files_no_version = [f.rsplit('_', 1)[0] for f in all_model_files]
all_model_files_no_version_unique = list(set(all_model_files_no_version))
all_model_files_no_version_unique
for model_file in all_model_files_no_version_unique:
    model_files = [f for f in all_model_files if f.startswith(model_file)]
    model_files.sort()
    latest_model_file = model_files[-1]
    print(latest_model_file)
    # delete all other versions
    for model_file_to_delete in model_files[:-1]:
        print('deleting ' + model_file_to_delete)
        os.remove('models/' + model_file_to_delete)
    # rename latest model file to v1
    file_ending = latest_model_file.rsplit('.', 1)[1]
    latest_model_file_rename = latest_model_file.rsplit('_', 1)[0] + '_v1' + '.' + file_ending
    if latest_model_file != latest_model_file_rename:
        print('renaming ' + latest_model_file + ' to ' + latest_model_file_rename)
        os.rename('models/' + latest_model_file, 'models/' + model_file + '_v1')

transformer_multivariate_quarter_hour_vis_columns_v1.csv
transformer_multivariate_quarter_hour_tmp_model_v1.pth
transformer_multivariate_half_hour_tmp_columns_v1.csv
transformer_multivariate_hour_vis_columns_v1.csv
transformer_model_tmp_v1.pth
transformer_multivariate_hour_hum_model_v1.pth
.DS_v1
renaming .DS_v1 to .DS_v1.DS_v1
transformer_multivariate_hour_vis_scaler_v1.pth
transformer_multivariate_hour_vis_model_v1.pth
transformer_multivariate_hour_tmp_columns_v1.csv
transformer_scaler_tmp_v1.pth
transformer_multivariate_half_hour_CO2_model_v1.pth
transformer_multivariate_half_hour_hum_scaler_v1.pth
transformer_multivariate_quarter_hour_CO2_model_v1.pth
transformer_multivariate_quarter_hour_hum_columns_v1.csv
transformer_multivariate_hour_hum_columns_v1.csv
transformer_multivariate_quarter_hour_CO2_scaler_v1.pth
transformer_multivariate_quarter_hour_vis_model_v1.pth
transformer_multivariate_hour_tmp_scaler_v1.pth
transformer_multivariate_half_hour_VOC_columns_v1.csv
transformer_multi

In [7]:
importlib.reload(utils)
model, scaler, rmse, mae, mape = utils.create_multivariate_transformer_model_for_feature(df, d_model=128, nhead=8, num_layers=4, dropout=0.1, batch_size=128, learning_rate=0.00025, epochs=2)

training data cutoff:  2023-07-15 02:45:00


/Users/timehmann/Library/Mobile Documents/com~apple~CloudDocs/Studium/Data_Science_Karlsruhe/Data_Science_Semester6/Internet_Of_Things/IoT_Project/utils.py:868: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feature_count = x[0].shape[1]
/Users/timehmann/Library/Mobile Documents/com~apple~CloudDocs/Studium/Data_Science_Karlsruhe/Data_Science_Semester6/Internet_Of_Things/IoT_Project/utils.py:869: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  print("feature count: ", feature_count)
/Users/timehmann/Library/Mob

tmp                         float64
hum                         float64
CO2                         float64
VOC                         float64
vis                         float64
IR                          float64
WIFI                        float64
BLE                         float64
rssi                        float64
channel_rssi                float64
channel_index               float64
spreading_factor            float64
bandwidth                   float64
f_cnt                       float64
isHoliday                   float64
isExamTime                  float64
weekday                     float64
month                       float64
hour_sin                    float64
semester_SS23               float64
semester_WS22/23            float64
semester_WS23/24            float64
group                       float64
device_id_hka-aqm-am001     float64
device_id_hka-aqm-am002     float64
device_id_hka-aqm-am003a    float64
device_id_hka-aqm-am003b    float64
device_id_hka-aqm-am004     

TypeError: 'NoneType' object cannot be interpreted as an integer

In [ ]:
importlib.reload(utils)
model, scaler = utils.create_multivariate_model_for_feature(df, aggregation_level='half_hour')

Training data shape: torch.Size([150050, 47]) torch.Size([150050])
Testing data shape: torch.Size([50023, 47]) torch.Size([50023])
     actual  predicted
0     408.0  477.67099
1     410.0  477.67099
2     406.5  477.67099
3     410.5  477.67099
4     413.5  477.67099
..      ...        ...
123   460.0  477.67099
124   440.0  477.67099
125   432.0  477.67099
126   432.5  477.67099
127   423.5  477.67099

[128 rows x 2 columns]
Epoch 1/1000, Validation Loss: 1.3423


KeyboardInterrupt: 

In [ ]:
train.columns

Index(['tmp', 'hum', 'CO2', 'VOC', 'vis', 'IR', 'WIFI', 'BLE', 'rssi',
       'channel_rssi', 'channel_index', 'spreading_factor', 'bandwidth',
       'f_cnt', 'isHoliday', 'isExamTime', 'weekday', 'month', 'hour_sin',
       'semester_SS23', 'semester_WS22/23', 'semester_WS23/24', 'tmp_lag_1',
       'tmp_lag_2', 'tmp_lag_3', 'tmp_lag_4', 'tmp_lag_5', 'hum_lag_1',
       'hum_lag_2', 'hum_lag_3', 'hum_lag_4', 'hum_lag_5', 'CO2_lag_1',
       'CO2_lag_2', 'CO2_lag_3', 'CO2_lag_4', 'CO2_lag_5', 'CO2_next',
       'VOC_lag_1', 'VOC_lag_2', 'VOC_lag_3', 'VOC_lag_4', 'VOC_lag_5',
       'vis_lag_1', 'vis_lag_2', 'vis_lag_3', 'vis_lag_4', 'vis_lag_5',
       'CO2_next_scaled'],
      dtype='object')

In [50]:
from datetime import datetime

importlib.reload(utils)
start_time = datetime.strptime('2022-11-03 10:00:00', '%Y-%m-%d %H:%M:%S')
predictions = utils.predict_data_multivariate_transformer(start_time=start_time, selected_room='am001', prediction_count=5)
predictions

loading latest scaler: models/transformer_multivariate_quarter_hour_CO2_scaler_v1.pth
loading latest model: models/transformer_multivariate_quarter_hour_CO2_model_v1.pth


/opt/homebrew/lib/python3.11/site-packages/sklearn/base.py:409: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/sklearn/base.py:409: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/sklearn/base.py:409: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/sklearn/base.py:409: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/sklearn/base.py:409: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/sklearn/base.py:409: UserWarning: X does not have valid f

,date_time_rounded,CO2_pred
0,2022-11-03 00:00:00,458.0
1,2022-11-03 00:15:00,457.0
2,2022-11-03 00:30:00,463.0
3,2022-11-03 00:45:00,462.0
4,2022-11-03 01:00:00,459.0
...,...,...
91,2022-11-03 22:45:00,461.0
92,2022-11-03 23:00:00,461.0
93,2022-11-03 23:15:00,461.0
94,2022-11-03 23:30:00,461.0


In [21]:
predictions.columns

Index(['date_time_rounded', 'tmp', 'hum', 'CO2', 'VOC', 'vis', 'IR', 'WIFI',
       'BLE', 'rssi', 'channel_rssi', 'channel_index', 'spreading_factor',
       'bandwidth', 'f_cnt', 'isHoliday', 'isExamTime', 'weekday', 'month',
       'hour_sin', 'semester_SS23', 'semester_WS22/23', 'semester_WS23/24',
       'group', 'device_id_hka-aqm-am001', 'device_id_hka-aqm-am002',
       'device_id_hka-aqm-am003a', 'device_id_hka-aqm-am003b',
       'device_id_hka-aqm-am004', 'device_id_hka-aqm-am005',
       'device_id_hka-aqm-am107', 'device_id_hka-aqm-am109',
       'device_id_hka-aqm-am110', 'device_id_hka-aqm-am111',
       'device_id_hka-aqm-am115', 'device_id_hka-aqm-am116',
       'device_id_hka-aqm-am117', 'device_id_hka-aqm-am123',
       'device_id_hka-aqm-am124', 'device_id_hka-aqm-am126',
       'device_id_hka-aqm-am201a', 'device_id_hka-aqm-am201b',
       'device_id_hka-aqm-am204', 'device_id_hka-aqm-am205',
       'device_id_hka-aqm-am209', 'device_id_hka-aqm-am210',
       'devi